# Data Generation

## Setup

In [1]:
import os

os.chdir("../../../")

In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

True

In [3]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.pydantic_v1 import BaseModel
from langchain_experimental.tabular_synthetic_data.openai import (
    OPENAI_TEMPLATE,
    create_openai_data_generator,
)
from langchain_experimental.tabular_synthetic_data.prompts import (
    SYNTHETIC_FEW_SHOT_PREFIX,
    SYNTHETIC_FEW_SHOT_SUFFIX,
)
from langchain_openai import ChatOpenAI

## Define Data Model

In [4]:
class MedicalBilling(BaseModel):
    patient_id: int
    patient_name: str
    diagnosis_code: str
    procedure_code: str
    total_charge: float
    insurance_claim_amount: float

## Sample Data

In [5]:
examples = [
    {
        "example": """Patient ID: 123456, Patient Name: John Doe, Diagnosis Code: 
        J20.9, Procedure Code: 99203, Total Charge: $500, Insurance Claim Amount: $350"""
    },
    {
        "example": """Patient ID: 789012, Patient Name: Johnson Smith, Diagnosis 
        Code: M54.5, Procedure Code: 99213, Total Charge: $150, Insurance Claim Amount: $120"""
    },
    {
        "example": """Patient ID: 345678, Patient Name: Emily Stone, Diagnosis Code: 
        E11.9, Procedure Code: 99214, Total Charge: $300, Insurance Claim Amount: $250"""
    },
]

## Craft a Prompt Template

The generator doesn't magically know how to create our data; we need to guide it. We do this by creating a prompt template. This template helps instruct the underlying language model on how to produce synthetic data in the desired format.

In [6]:
OPENAI_TEMPLATE = PromptTemplate(input_variables=["example"], template="{example}")

prompt_template = FewShotPromptTemplate(
    prefix=SYNTHETIC_FEW_SHOT_PREFIX,
    examples=examples,
    suffix=SYNTHETIC_FEW_SHOT_SUFFIX,
    input_variables=["subject", "extra"],
    example_prompt=OPENAI_TEMPLATE
)

## Creating the Data Generator

In [8]:
synthetic_data_generator = create_openai_data_generator(
    output_schema=MedicalBilling,
    llm=ChatOpenAI(
        model="gpt-4o-mini",
        temperature=1
    ),  # You'll need to replace with your actual Language Model instance
    prompt=prompt_template,
)

## Generate Synthetic Data

In [9]:
synthetic_results = synthetic_data_generator.generate(
    subject="medical_billing",
    extra="the name must be chosen at random. Make it something you wouldn't normally choose.",
    runs=10,
)

In [10]:
synthetic_results

[MedicalBilling(patient_id=864209, patient_name='Zephyr Ainsworth', diagnosis_code='H25.9', procedure_code='99202', total_charge=450.0, insurance_claim_amount=300.0),
 MedicalBilling(patient_id=582349, patient_name='Quasar Nebular', diagnosis_code='J45.909', procedure_code='99203', total_charge=220.0, insurance_claim_amount=180.0),
 MedicalBilling(patient_id=745389, patient_name='Fibonacci Seahorse', diagnosis_code='M54.4', procedure_code='99205', total_charge=370.0, insurance_claim_amount=320.0),
 MedicalBilling(patient_id=682174, patient_name='Celestial Pumpkin', diagnosis_code='I10', procedure_code='99204', total_charge=525.0, insurance_claim_amount=450.0),
 MedicalBilling(patient_id=236178, patient_name='Galactic Wombat', diagnosis_code='E11.9', procedure_code='99202', total_charge=150.0, insurance_claim_amount=120.0),
 MedicalBilling(patient_id=937485, patient_name='Quantum Raspberry', diagnosis_code='J45.909', procedure_code='99213', total_charge=230.0, insurance_claim_amount=190